In [21]:
run_folders = ['LiftMP_runs']
relevant_keys = [ 'model_type', 'num_layers',
       'repeat_lift_layers', 'num_layers_project', 'rank', 'vc_penalty', 
       'dataset', 'infinite',  'positional_encoding', 'pe_dimension',
       'max_valid_score', 'max_valid_epoch', 'gen_n'
       'scores']

In [22]:
import json
import pandas as pd
import numpy as np
import os

folder_path = '/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs'
model_list = [os.path.join(folder_path, run_folder, x) for run_folder in run_folders for x in os.listdir(os.path.join(folder_path, run_folder))  ]


rows = []

errored = []
for model_folder in model_list:
    try:
        with open(os.path.join(model_folder, 'params.txt'), 'r') as f:
            model_args = json.load(f)
        
        losses = np.load(os.path.join(model_folder, 'valid_scores.npy'))
        test_losses = np.load(os.path.join(model_folder, 'test_scores.npy'))
        modeldict = model_args #{x: model_args[x] for x in params}
        #modeldict['max_valid_score'] = max(losses)
        #modeldict['max_valid_epoch'] = np.argmax(losses)
        modeldict['scores'] = test_losses[np.argmax(losses)]
        modeldict['baseline'] = False
        modeldict['full_scores'] = losses
        modeldict['valid_scores'] = losses
        modeldict['test_scores'] = test_losses
        modeldict['model_folder'] = model_folder

        rows.append(modeldict)
        
    except:
        print(f'something is wrong w/ {model_folder}')
        errored.append(model_folder)

something is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:d7295454a183410793f153014e74c48fb81051635748e2fa4358e5e62ff747c8
something is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:959f4a30685c103490271cfaa513d24bffe58e4ff7f56301797ac56a2ab76978
something is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:af7aa021f2876f5b35b4888f53a3bc63a8846a1ae09ca8bd2d6f8c1ee2f9923a
something is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:e50ba9e1678781fa6302c1c2e627dc630da420c987ab9c613ab6878be6bb9af3
something is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:55bb40501c2d33a9f61b4f08f82ad78379330f60679339affb7ac8405e3dbd0a
something is wrong w/ /home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:ac400b21989cc7b2bcbb8be98668cb4e2807c536a5eb86be327466214f4df1c5
something is wrong w/ /home/bcjexu

In [38]:
df = pd.DataFrame.from_records(rows)

In [39]:
df.gen_n = df.gen_n.apply(lambda x: min(x) if isinstance(x,list) else x)

In [52]:
Counter(df.dataset)

Counter({'ErdosRenyi': 24,
         'WattsStrogatz': 24,
         'BarabasiAlbert': 24,
         'PowerlawCluster': 24,
         'IMDB-BINARY': 16,
         'ENZYMES': 16,
         'MUTAG': 16,
         'PROTEINS': 16,
         'COLLAB': 12})

In [40]:
bestdf = df.loc[df.groupby(['dataset', 'gen_n'])['scores'].idxmax()]#[relevant_keys]

In [51]:
bestdf[['dataset', 'gen_n', 'positional_encoding', 'pe_dimension']]

,dataset,gen_n,positional_encoding,pe_dimension
52,BarabasiAlbert,50,random_walk,8
155,BarabasiAlbert,100,random_walk,8
150,BarabasiAlbert,400,random_walk,4
36,COLLAB,100,random_walk,4
2,ENZYMES,100,random_walk,8
13,ErdosRenyi,50,random_walk,4
59,ErdosRenyi,100,random_walk,4
171,ErdosRenyi,400,random_walk,8
24,IMDB-BINARY,100,random_walk,16
7,MUTAG,100,random_walk,8


In [42]:
from collections import Counter
Counter(df.dataset)

Counter({'ErdosRenyi': 24,
         'WattsStrogatz': 24,
         'BarabasiAlbert': 24,
         'PowerlawCluster': 24,
         'IMDB-BINARY': 16,
         'ENZYMES': 16,
         'MUTAG': 16,
         'PROTEINS': 16,
         'COLLAB': 12})

In [47]:
er = bestdf[bestdf['dataset'] == 'ErdosRenyi']
for x in er.model_folder:
    print(f"\"{os.path.join(x, 'best_model.pt')}\"")

"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:c259ee991d44601a24495085d478929b226e94ca5ffc2a75ab739957d94da4b0/best_model.pt"
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:9ede9cd33a4971c3246500914847936dbc9690bd4c05a450cdef0d6146b815f4/best_model.pt"
"/home/bcjexu/maxcut-80/bespoke-gnn4do/training_runs/LiftMP_runs/paramhash:631e9afcf3bd0330c4f9625392cc0b183926a43b2ba3d8c5d8fbaf91c779dc90/best_model.pt"
